In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download santander-customer-satisfaction

  0% 0.00/8.25M [00:00<?, ?B/s]
100% 8.25M/8.25M [00:00<00:00, 100MB/s]


In [ ]:
! unzip santander-customer-satisfaction.zip

Archive:  santander-customer-satisfaction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
! pip install h2o

     |████████████████████████████████| 177.0 MB 20 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=b089b3bb0421b49fe0202f537b553a9d0f86874edd46a96eb8461ab2431875af
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [ ]:
import h2o
import pandas as pd
import numpy as np
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0ua9kcnw
  JVM stdout: /tmp/tmp0ua9kcnw/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp0ua9kcnw/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_unknownUser_2d31gq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
df = h2o.import_file('train.csv', destination_frame = 'train')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
df1 = h2o.import_file('test.csv', destination_frame = 'test')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
y = 'TARGET'
X = df.col_names[1:-1]
X_test = df1.col_names[1:]

In [ ]:
# Because D is categorical
df['TARGET'] = df['TARGET'].asfactor()
df['TARGET'].levels()

[['0', '1']]

In [ ]:
gbm_params = {'learn_rate': np.linspace(0.0001, 1, 5).tolist(),
               'max_depth': [3, 5, 7],
               'ntrees': [50, 100]}
gbm = H2OGradientBoostingEstimator(distribution = "bernoulli")
gbm_grid = H2OGridSearch(model = gbm, grid_id = 'gbm_grid', hyper_params = gbm_params)
gbm_grid.train(x=X, y=y, training_frame = df, seed = 2022)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
      learn_rate  max_depth  ntrees          model_ids   logloss
0       0.250075        7.0   100.0  gbm_grid_model_27  0.107282
1       0.250075        7.0    50.0  gbm_grid_model_12  0.112599
2       0.500050        7.0   100.0  gbm_grid_model_28  0.115165
3       0.500050        7.0    50.0  gbm_grid_model_13  0.116138
4       0.250075        5.0   100.0  gbm_grid_model_22  0.117101
5       0.500050        5.0   100.0  gbm_grid_model_23  0.117581
6       0.500050        5.0    50.0   gbm_grid_model_8  0.121254
7       0.250075        5.0    50.0   gbm_grid_model_7  0.122130
8       0.500050        3.0   100.0  gbm_grid_model_18  0.124868
9       0.750025        7.0   100.0  gbm_grid_model_29  0.126144
10      0.250075        3.0   100.0  gbm_grid_model_17  0.127896
11      0.500050        3.0    50.0   gbm_grid_model_3  0.130226
12      0.750025        3.0    50.0   gbm_grid_model_4  0.13

In [ ]:
gcv_list = gbm_grid.get_grid(sort_by='auc',decreasing=True)
best_gcv = gcv_list.models[0]

In [ ]:
y_pred = best_gcv.predict(test_data = df1)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [ ]:
y_pred_df = y_pred.as_data_frame()

In [ ]:
submit = pd.concat([df1['ID'].as_data_frame(), y_pred_df[['p1']]], axis = 1)
submit.rename(columns = {'p1': 'TARGET'}, inplace = True)

In [ ]:
submit.to_csv('submit-21-05-GBM.csv', index=False)

Score: 0.80911

Public score: 0.82673

In [ ]:
h2o.cluster().shutdown()

H2O session _sid_8dc7 closed.
